In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=1000

In [2]:
from featurewiz import LazyTransformer, SuloRegressor, SuloClassifier
from featurewiz import FeatureWiz, cross_val_model_predictions, get_class_weights
from featurewiz import print_regression_metrics, print_classification_metrics

Imported lazytransform v1.11. 

Imported featurewiz 0.5.3. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True, scalers="std",
        		category_encoders="auto", add_missing=False, verbose=0. imbalanced=False,
        		ae_options={})
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [3]:
trainfile = 'c:/users/ram/documents/ram/data_sets/kaggle/diabetes.csv'
datapath = '../Ram/Data_Sets/'
filename = 'winequality.csv'
filename = 'affairs.csv'
trainfile = datapath+filename
sep = ','
dft = pd.read_csv(trainfile,sep=sep)
dft.drop(['affair', 'affairs'],axis=1, inplace=True)
print(dft.shape)
dft.head()

(6366, 9)


,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair_multiclass
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4


In [4]:
target = 'affair_multiclass'
modeltype = 'Multi_Classification'
preds = [x for x in list(dft) if x not in target]
dft[target].nunique()

7

In [5]:
from sklearn.model_selection import train_test_split
from featurewiz import FE_kmeans_resampler
if modeltype == 'Regression':
    X_train, X_test, y_train, y_test = train_test_split(dft[preds], dft[target], test_size=0.20, random_state=1,)
    #X_train_over, y_train_over = FE_kmeans_resampler(X_train, y_train, target, smote='',verbose=0)
    #print(X_train_over.shape, X_test.shape)
    #train, test = pd.concat([X_train_over, pd.Series(y_train_over,name=target)], axis=1), pd.concat([X_test, y_test], axis=1)
    train, test = train_test_split(dft, test_size=0.20, random_state=42)
else:
    X_train, X_test, y_train, y_test = train_test_split(dft[preds], dft[target], test_size=0.20, 
                                                    stratify=dft[target],
                                                    random_state=42)
    train, test = train_test_split(dft, test_size=0.20, random_state=42,
                                                    stratify=dft[target]
                                                   )
print(X_train.shape, X_test.shape)

(5092, 8) (1274, 8)


In [6]:
transform_target = True

In [7]:
fwiz = FeatureWiz(corr_limit=0.80, auto_encoders='dae', feature_engg=['interactions', 'poly2'], 
                  category_encoders='auto', scalers='MinMax',
                 dask_xgboost_flag=False, nrows=None, verbose=2, skip_sulov=False,
                 skip_xgboost=False, transform_target=transform_target,
                 ae_options={}) 
X_train_selected, y_train = fwiz.fit_transform(X_train, y_train) 

featurewiz is given 0.8 as correlation limit...
    final list of feature engineering given: ['interactions', 'poly2']
    final list of category encoders given: ['onehot', 'label']
Since Auto Encoders are selected for feature extraction,
    SULOV is skipped...
    Recursive XGBoost is skipped...
DenoisingAutoEncoder()
    AE dictionary given: dict_items([])
    final list of scalers given: [minmax]
Loaded input data. Shape = (5092, 8)
    selecting 8 numeric features for further processing...
#### Starting featurewiz transform for train data ####
No groupby features created since no groupby feature engg specified
No interactions created for categorical vars since number less than 2
    Single_Label Multi_Classification problem 
Shape of dataset: (5092, 44). Now we classify variables into different types...
    Returning dictionary for variable types with following keys:
                        continuous_vars = 44, int_vars = 0, 
                        discrete_string_vars = 0, nlp_

255/255 [==============================] - 1s 5ms/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.0189 - val_mse: 0.0189
Epoch 42/100
255/255 [==============================] - 1s 5ms/step - loss: 0.0255 - mse: 0.0255 - val_loss: 0.0189 - val_mse: 0.0189
Epoch 43/100
255/255 [==============================] - 1s 5ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.0184 - val_mse: 0.0184
Epoch 44/100
255/255 [==============================] - 1s 5ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.0191 - val_mse: 0.0191
Fitting and transforming an Auto Encoder for dataset...
Shape of transformed data due to auto encoder = (5092, 5)
    Single_Label Multi_Classification problem 
    Dropping 1 columns due to zero variance: Index(['ae_feature_5'], dtype='object')
    time taken to run entire featurewiz = 173 second(s)
Recursive XGBoost selected 4 features...


In [8]:
X_test_selected = fwiz.transform(X_test) 

#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (1274, 8)
    Beware! feature_engg will add 100s, if not 1000s of additional features to your dataset!
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 1 second(s)
    Shape of transformed dataset: (1274, 44)
Shape of transformed data due to auto encoder = (1274, 5)
    Dropping 1 columns due to zero variance: Index(['ae_feature_5'], dtype='object')
Returning dataframe with 4 features 


In [9]:
if transform_target:
    y_test = fwiz.lazy.yformer.transform(y_test)

In [10]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, classification_report
from featurewiz import get_class_distribution
# Updating the Random Forest Classifier with the corrected class weights
if modeltype == 'Regression':
    rf_classifier = RandomForestRegressor(random_state=42)
else:
    # Correctly computing class weights for the classes present in the training set
    class_weights_dict_corrected = get_class_distribution(y_train)
    rf_classifier = RandomForestClassifier(n_estimators=100, class_weight=class_weights_dict_corrected, random_state=42)


# Fitting the classifier on the training data
rf_classifier.fit(X_train_selected, y_train)

# Predicting on the test set
y_pred = rf_classifier.predict(X_test_selected)

if modeltype == 'Regression':
    print_regression_metrics(y_test, y_pred, verbose=1)
else:
    # Evaluating the classifier
    y_probas = rf_classifier.predict_proba(X_test_selected)
    print_classification_metrics(y_test, y_pred, y_probas, verbose=1)

Bal accu 15%
ROC AUC = 0.64
              precision    recall  f1-score   support

           0       0.83      0.95      0.88      1050
           1       0.10      0.03      0.05        86
           2       0.17      0.04      0.07        48
           3       0.00      0.00      0.00        26
           4       0.15      0.06      0.08        35
           5       0.00      0.00      0.00        17
           6       0.00      0.00      0.00        12

    accuracy                           0.79      1274
   macro avg       0.18      0.15      0.16      1274
weighted avg       0.70      0.79      0.74      1274

final average balanced accuracy score = 0.15


In [11]:
from featurewiz import IterativeDoubleClassifier,  IterativeSearchClassifier, IterativeBestClassifier


In [12]:
from xgboost import XGBClassifier, XGBRFRegressor

In [13]:
from featurewiz.blagging import BlaggingClassifier

In [14]:
# Usage example
import time
start_time = time.time()
early = BlaggingClassifier(n_estimators=200)
late = RandomForestClassifier(n_estimators=100)
base = XGBClassifier(n_estimators=200)
#sdae =  IterativeBestClassifier(base_estimator=late)
#sdae = IterativeSearchClassifier(base_estimator1=early, base_estimator2=late)
sdae = IterativeDoubleClassifier(base_estimator1=early, base_estimator2=late, weights={1: 0.6, 2: 0.4} )
sdae.fit(X_train_selected, y_train)
print('Time taken = %0.0f seconds' %(time.time()-start_time))

Time taken = 12 seconds


In [15]:
y_pred = sdae.predict(X_test_selected)

In [16]:
if modeltype == 'Regression':
    print_regression_metrics(y_test, y_pred, verbose=1)
else:
    # Evaluating the classifier
    y_probas = sdae.predict_proba(X_test_selected)
    print_classification_metrics(y_test, y_pred, y_probas, verbose=1)

Bal accu 18%
ROC AUC = 0.70
              precision    recall  f1-score   support

           0       0.84      0.83      0.84      1050
           1       0.10      0.09      0.10        86
           2       0.08      0.08      0.08        48
           3       0.07      0.12      0.09        26
           4       0.06      0.06      0.06        35
           5       0.06      0.06      0.06        17
           6       0.00      0.00      0.00        12

    accuracy                           0.70      1274
   macro avg       0.17      0.18      0.17      1274
weighted avg       0.71      0.70      0.70      1274

final average balanced accuracy score = 0.18
